In [33]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
import os
import requests
import urllib

In [35]:
source = requests.get('https://au.indeed.com/jobs?q=data+scientist&l=Australia').text

In [36]:
soup = BeautifulSoup(source, 'lxml')

In [38]:
#pretiffy gives the results with HTML indentation to help with presentation of the output
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en">
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <script type="text/javascript">
   (function ( tk ) { if ( tk && document.images ) { var s="/", q="?", a="&", e="="; (new Image()).src = s+"rpc"+s+"log"+q+"a"+e+"jslat"+a+"tk"+e+tk; } })('1daaknmk580qr803');
  </script>
  <script src="/s/6f6b3d3/en_AU.js" type="text/javascript">
  </script>
  <link href="/s/97464e7/jobsearch_all.css" rel="stylesheet" type="text/css"/>
  <link href="http://au.indeed.com/rss?q=data+scientist&amp;l=Australia" rel="alternate" title="Data Scientist Jobs in Australia" type="application/rss+xml"/>
  <link href="/m/jobs?q=data+scientist&amp;l=Australia" media="only screen and (max-width: 640px)" rel="alternate"/>
  <link href="/m/jobs?q=data+scientist&amp;l=Australia" media="handheld" rel="alternate"/>
  <script type="text/javascript">
   if (typeof window['closureReadyCallbacks'] == 'undefined') {
        window['closureReadyCallbacks'] =

In [39]:
#must include an underscore after 'class' in the BS parameters because 'class' is a reserved command
#.find gives just one result (.find)
jobs = soup.find(class_='result')
#this output will include the the main HTML elements of the first job posting
print(jobs.prettify())

<div class="jobsearch-SerpJobCard unifiedRow row result" data-jk="201ba8055b3c8390" data-tn-component="organicJob" data-tu="" id="p_201ba8055b3c8390">
 <div class="title">
  <a class="jobtitle turnstileLink " data-tn-element="jobTitle" href="/rc/clk?jk=201ba8055b3c8390&amp;fccid=017050e537d7779b&amp;vjs=3" id="jl_201ba8055b3c8390" onclick=" setRefineByCookie([]); return rclk(this,jobmap[0],true,1);" onmousedown="return rclk(this,jobmap[0],1);" rel="noopener nofollow" target="_blank" title="Student Data Scientist (APS 3)">
   Student
   <b>
    Data
   </b>
   <b>
    Scientist
   </b>
   (APS 3)
  </a>
 </div>
 <div class="sjcl">
  <div>
   <span class="company">
    <a class="turnstileLink" data-tn-element="companyName" href="/cmp/Australian-Bureau-of-Statistics" onmousedown="this.href = appendParamsOnce(this.href, 'from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=201ba8055b3c8390&amp;jcid=017050e537d7779b')" rel="noopener" target="_blank">
     Australian Bureau of Statistics

In [42]:
#Now, just get the Job Title (the job title is wrapped in an 'a' tag)
job_title = jobs.a.text.strip()
print(job_title)

Student Data Scientist (APS 3)


In [43]:
#Now get the employer
employer = jobs.span.text.strip()
print(employer)

Australian Bureau of Statistics


In [44]:
#the span tag is already taken, so I want to find the specific 'span' that has the class='location'
location = jobs.find('span', class_='location').text.strip()
print(location)

Australia


In [47]:
#salary, if present
salary = jobs.find('span', class_='salary no-wrap').text.strip()
print(salary)

$34.64 an hour


In [48]:
#text summary of the job posting
summary = jobs.find('div', class_='summary').text.strip()
print(summary)

The ABS is currently seeking Student Data Scientists in our Statistical Data Integration Division in the Canberra office. Our Student Data Scientists will work with data under the supervision of experienced team members to understand large...


In [49]:
date = jobs.find('span', class_='date').text.strip()
print(date)

9 days ago


In [50]:
#Next, I'll loop through every job posting return the above variables (using .find_all) - commented out after testing
# for jobs in soup.find_all(class_='result'):
#     print(jobs.prettify())
#     print('--------------')
#This code will show the HTML output for all jobs in the search, separated by '------' for each job posting on this page

<div class="jobsearch-SerpJobCard unifiedRow row result" data-jk="201ba8055b3c8390" data-tn-component="organicJob" data-tu="" id="p_201ba8055b3c8390">
 <div class="title">
  <a class="jobtitle turnstileLink " data-tn-element="jobTitle" href="/rc/clk?jk=201ba8055b3c8390&amp;fccid=017050e537d7779b&amp;vjs=3" id="jl_201ba8055b3c8390" onclick=" setRefineByCookie([]); return rclk(this,jobmap[0],true,1);" onmousedown="return rclk(this,jobmap[0],1);" rel="noopener nofollow" target="_blank" title="Student Data Scientist (APS 3)">
   Student
   <b>
    Data
   </b>
   <b>
    Scientist
   </b>
   (APS 3)
  </a>
 </div>
 <div class="sjcl">
  <div>
   <span class="company">
    <a class="turnstileLink" data-tn-element="companyName" href="/cmp/Australian-Bureau-of-Statistics" onmousedown="this.href = appendParamsOnce(this.href, 'from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=201ba8055b3c8390&amp;jcid=017050e537d7779b')" rel="noopener" target="_blank">
     Australian Bureau of Statistics

In [53]:
#Now I want to execute the code to return the clean variables and to not break if certain variables aren't present
for jobs in soup.find_all(class_='result'):
    #first, try to return the value if present and print it out; if not, return nothing
    try:
        job_title = jobs.a.text.strip()
    except Exception as e:
        job_title = None
    print('Job Title: ', job_title)
    
    try:
        employer = jobs.span.text.strip()
    except Exception as e:
        employer = None
    print('Employer: ', employer)
    
    try:
        location = jobs.find('span', class_='location').text.strip()
    except Exception as e:
        location = None
    print('Location: ', location)
    
    try:
        salary = jobs.find('span', class_='salary no-wrap').text.strip()
    except Exception as e:
        salary = None
    print('Salary: ', salary)
    
    try:
        summary = jobs.find('div', class_='summary').text.strip()
    except Exception as e:
        summary = None
    print('Summary: ', summary)
    
    try:
        date = jobs.find('span', class_='date').text.strip()
    except Exception as e:
        date = None
    print('Date: ', date)
    
    print('-------------')#seperate postings

Job Title:  Student Data Scientist (APS 3)
Employer:  Australian Bureau of Statistics
Location:  Australia
Salary:  $34.64 an hour
Summary:  The ABS is currently seeking Student Data Scientists in our Statistical Data Integration Division in the Canberra office. Our Student Data Scientists will work with data under the supervision of experienced team members to understand large...
Date:  9 days ago
-------------
Job Title:  Data Scientist
Employer:  Accenture
Location:  Sydney NSW
Salary:  None
Summary:  Contribute to the team’s effort for building, nurturing and maintaining a leading-edge data science team. 2+ years’ hands-on experience in gathering, curating and preparing appropriate representative data for building machine learning and...
Date:  30+ days ago
-------------
Job Title:  Senior Data Scientist
Employer:  Queensland Government
Location:  Brisbane QLD
Salary:  None
Summary:  Experience working as a data scientist, data analyst, data engineer, transport modeller, or similar

In [67]:
#Next step is to collect jobs over several pages
#URL adds '&start=10' for second page and goes up in 10 increments (e.g. page 5 = '&start=40')
#So, need to create a list at the top of the script with the number of pages I want included in the scrape
#Here's the full code to execute:

#Also, importing time enables delayed requests to prevent being blocked on certain websites (time.sleep is called below)
import time
#I'll write this to a csv
import csv
import os

max_results = 2000

i = 0    

pages = []

#save this output data as a csv file in the specified location (the 'a' means append and f_output = file output)
with open('/Users/nikolasdawson/Documents/Personal_Projects/Machine_Learning/indeed_webscraping/indeed_test_scrape.csv', 'a', encoding='utf-8', newline='') as f_output:
    csv_print = csv.writer(f_output)
    
    #check whether the file has some memory in it
    file_is_empty = os.stat('/Users/nikolasdawson/Documents/Personal_Projects/Machine_Learning/indeed_webscraping/indeed_test_scrape.csv').st_size == 0
    if file_is_empty:
        csv_print.writerow(['CleanJobTitle', 'Employer', 'Location', 'Salary', 'Summary', 'Date'])

    for page in range(0, max_results, 10):
        #the below source is the same as above, except it has the page reference at the end
        #I've added {} and .format(page) so that it can reference the pages list and eventually append and loop 
        source = requests.get('https://au.indeed.com/jobs?q=data+scientist&l=Australia&start={}'.format(page)).text

        soup = BeautifulSoup(source, 'lxml')

        for jobs in soup.find_all(class_='result'):
        #first, try to return the value if present and print it out; if not, return nothing
            try:
                job_title = jobs.a.text.strip()
            except Exception as e:
                job_title = None
            print('Job Title: ', job_title)

            try:
                employer = jobs.span.text.strip()
            except Exception as e:
                employer = None
            print('Employer: ', employer)

            try:
                location = jobs.find('span', class_='location').text.strip()
            except Exception as e:
                location = None
            print('Location: ', location)

            try:
                salary = jobs.find('span', class_='salary no-wrap').text.strip()
            except Exception as e:
                salary = None
            print('Salary: ', salary)

            try:
                summary = jobs.find('div', class_='summary').text.strip()
            except Exception as e:
                summary = None
            print('Summary: ', summary)

            try:
                date = jobs.find('span', class_='date').text.strip()
            except Exception as e:
                date = None
            print('Date: ', date)
            
            i += 1

            csv_print.writerow([job_title, employer, location, salary, summary, date])
            
            print('-------------')

            #will delay requests for X seconds per page
            time.sleep(0.5)

Job Title:  Student Data Scientist (APS 3)
Employer:  Australian Bureau of Statistics
Location:  Australia
Salary:  $34.64 an hour
Summary:  The ABS is currently seeking Student Data Scientists in our Statistical Data Integration Division in the Canberra office. Our Student Data Scientists will work with data under the supervision of experienced team members to understand large...
Date:  9 days ago
-------------
Job Title:  Data Scientist
Employer:  Zendesk
Location:  Melbourne VIC
Salary:  None
Summary:  The Data Science Team, together with Data Engineering, have built Satisfaction Prediction, Answer Bot and Content Cues — products that use Machine Learning (ML) and Artificial Intelligence (AI) to help our customers work more efficiently....
Date:  1 hour ago
-------------
Job Title:  Senior Data Scientist
Employer:  Queensland Government
Location:  Brisbane QLD
Salary:  None
Summary:  Experience working as a data scientist, data analyst, data engineer, transport modeller, or similar 

Job Title:  Senior Data Scientist
Employer:  Foxtel
Location:  Moonee Ponds VIC
Salary:  None
Summary:  An Advanced user of data visualisation techniques to tell the story of data. Expert in data preparation and manipulation. Lead the delivery of Data Science projects. Proactively create new and innovative Data Science services....
Date:  26 days ago
-------------
Job Title:  Senior Data Scientist
Employer:  Sportsbet.com.au
Location:  Melbourne VIC
Salary:  None
Summary:  We are looking for an experienced Data Scientist to bring our Machine Learning capability to the next level. Skills in Data engineering and/or Software engineering are a definite plus. In the Data Science team we believe in using Advanced...
Date:  30+ days ago
-------------
Job Title:  Data Scientist/Data Analyst
Employer:  EnergyAustralia
Location:  Victoria
Salary:  None
Summary:  Data Scientist/Data Analyst. Strong data analytics skills (essential). In the role of Data Scientist your primary objectives are to lea

Job Title:  Lead Data Scientist
Employer:  BCG Digital Ventures
Location:  Sydney NSW
Salary:  None
Summary:  As a Lead Data Scientist, you will:. As a Lead Data Scientist, you will need:. Experience leading and managing a team of data scientists. Knowledge in Big Data and Fast Data (i.e.:....
Date:  6 months ago
-------------
Job Title:  Data Scientist
Employer:  Launch Recruitment
Location:  Melbourne VIC
Salary:  $700 - $900 a day
Summary:  Designing and building data pipelines and data streams. Knowledge and exposure to various Azure Cloud Data Services and Microsoft data technologies. Our client who is a well-established leading provider of innovative digital cloud services...
Date:  7 days ago
-------------
Job Title:  Data Scientist
Employer:  Opus Recruitment Solutions
Location:  Sydney NSW
Salary:  $300 - $500 a day
Summary:  Junior Data Scientist. Developing solutions to business problems by using all aspects of data science, machine learning, data visualisation and data engi

Job Title:  Research Assistant / Life Scientist ACBD
Employer:  Alfred Health
Location:  Melbourne VIC
Salary:  None
Summary:  Medical Scientist Grade 1 ($56,976.40 - $81,723.20). For grant applications, research and conference papers) requiring the conducting of literature reviews and...
Date:  26 days ago
-------------
Job Title:  IT Data Scientist
Employer:  Peoplebank
Location:  Melbourne VIC
Salary:  None
Summary:  Work with a cross functional team of data scientists, data analysts, report developers and engineers to move Proof-of-Concept models/products to production for...
Date:  8 days ago
-------------
Job Title:  Application Scientist (EM, Life Science, Melbourne)
Employer:  Thermo Fisher Scientific
Location:  Melbourne VIC
Salary:  None
Summary:  Low dose under Cryo condition and automatic data collection using EPU or other third party software(SerialEM)....
Date:  26 days ago
-------------
Job Title:  Environmental Engineer/Scientist
Employer:  Golder Associates
Location:  

Job Title:  Research Fellow - Molecular and Cell Biology
Employer:  Monash University
Location:  Clayton VIC
Salary:  $68,040 - $92,343 a year
Summary:  The successful candidate will have access to world-class research platforms at Monash University and High-Performance Computing (HPC) server for data analysis....
Date:  26 days ago
-------------
Job Title:  Research and Development Manager, Formulation Scientist
Employer:  Parnell Veterinary Pharmaceuticals
Location:  Alexandria NSW
Salary:  None
Summary:  At least 3 years’ experience as a supervisor, preferably of experienced laboratory scientists. Review of data generated by the R&D associate following in house...
Date:  11 days ago
-------------
Job Title:  Senior Environmental Scientist
Employer:  RPS Group
Location:  Cairns QLD
Salary:  None
Summary:  High level of data assessment and analysis. Willingness to oversee and provide support to middle level environment scientists....
Date:  30+ days ago
-------------
Job Title:  EMPTY

...

Job Title:  Azure Architect
Employer:  Beacham Group Pty Ltd
Location:  Perth WA
Salary:  None
Summary:  Strong experience specifically in Azure data storage, integration and data virtualisation components sufficient to recommend components, define governance and...
Date:  7 days ago
-------------
Job Title:  Senior Lead Developer (Contract) - $1000pd
Employer:  CoTalent
Location:  Sydney NSW
Salary:  None
Summary:  Collaborate with Developers, Academics and Data Scientists. Our client is a pioneering, internationally operating organisation, which is a true global leader in...
Date:  2 days ago
-------------
Job Title:  Senior Software Engineer | Python & Machine Learning
Employer:  Adaptive Resourcing
Location:  Perth WA
Salary:  None
Summary:  Python, shell scripting, database management, data analysis and visualization, Keras. Every now and then I meet with a company that has so much potential that I...
Date:  20 days ago
-------------
Job Title:  DevOps Engineer
Employer:  Professi

Job Title:  Lead Statistical Programmer
Employer:  Clinical Professionals
Location:  Mainland WA
Salary:  None
Summary:  Evaluate different analysis methodology and data presentation techniques. A deep understanding of clinical study designs and summarization of data and...
Date:  30+ days ago
-------------
Job Title:  Senior Machine Learning Engineer - Apache Spark + AWS - 6 Mo...
Employer:  Talent Insights Group
Location:  Sydney NSW
Salary:  None
Summary:  Experience with design or improvement of distributed computing solutions – so very likely helping data scientists & modelling teams distribute computation...
Date:  30+ days ago
-------------
Job Title:  Senior Pre Sales Consultant
Employer:  CITI Recruitment
Location:  Sydney NSW
Salary:  None
Summary:  Software Engineer or Data Scientist background. Knowledge of Service Oriented Architectures, Microservices, Big Data, and IoT principles....
Date:  9 days ago
-------------
Job Title:  Azure Architect
Employer:  Beacham Group Pty 

Job Title:  PreSales Consultant | AI | Machine Learning | Advanced Analy...
Employer:  Eagna Consulting
Location:  Melbourne VIC
Salary:  None
Summary:  My client is a global leader in the AI, Analytics and Statistical space with over 300 staff in Australia. They are leading the way in new technology and...
Date:  30+ days ago
-------------
Job Title:  Senior DevOps Engineer
Employer:  Onset Group Pty Ltd
Location:  Sydney NSW
Salary:  None
Summary:  Collaboration with top shelf Data Engineers and Data Scientists across the business will be frequent, so an appetite to be challenged and learn is crucial....
Date:  30+ days ago
-------------
Job Title:  Lead Statistical Programmer
Employer:  Clinical Professionals
Location:  Mainland WA
Salary:  None
Summary:  Evaluate different analysis methodology and data presentation techniques. A deep understanding of clinical study designs and summarization of data and...
Date:  30+ days ago
-------------
Job Title:  Senior Machine Learning Enginee

Job Title:  PreSales Consultant | AI | Machine Learning | Advanced Analy...
Employer:  Eagna Consulting
Location:  Melbourne VIC
Salary:  None
Summary:  My client is a global leader in the AI, Analytics and Statistical space with over 300 staff in Australia. They are...
Date:  30+ days ago
-------------
Job Title:  Senior DevOps Engineer
Employer:  Onset Group Pty Ltd
Location:  Sydney NSW
Salary:  None
Summary:  Collaboration with top shelf Data Engineers and Data Scientists across the business will be frequent, so an appetite to...
Date:  30+ days ago
-------------
Job Title:  Lead Statistical Programmer
Employer:  Clinical Professionals
Location:  Mainland WA
Salary:  None
Summary:  Evaluate different analysis methodology and data presentation techniques....
Date:  30+ days ago
-------------
Job Title:  Senior Machine Learning Engineer - Apache Spark + AWS - 6 Mo...
Employer:  Talent Insights Group
Location:  Sydney NSW
Salary:  None
Summary:  Experience with design or improvement o

KeyboardInterrupt: 

In [ ]:
## This is the previous iteration of the code where you specify the contents of the list with the incrementer

# #Next step is to collect jobs over several pages
# #URL adds '&start=10' for second page and goes up in 10 increments (e.g. page 5 = '&start=40')
# #So, need to create a list at the top of the script with the number of pages I want included in the scrape
# #Here's the full code to execute:

# #Also, importing time enables delayed requests to prevent being blocked on certain websites (time.sleep is called below)
# import time
# #I'll write this to a csv
# import csv
# import os

# pages = [10, 20, 30, 40, 50]

# #save this output data as a csv file in the specified location (the 'a' means append and f_output = file output)
# with open('/Users/nikolasdawson/Documents/Personal_Projects/Machine_Learning/indeed_webscraping/indeed_test_scrape.csv', 'a', encoding='utf-8', newline='') as f_output:
#     csv_print = csv.writer(f_output)
    
#     #check whether the file has some memory in it
#     file_is_empty = os.stat('/Users/nikolasdawson/Documents/Personal_Projects/Machine_Learning/indeed_webscraping/indeed_test_scrape.csv').st_size == 0
#     if file_is_empty:
#         csv_print.writerow(['CleanJobTitle', 'Employer', 'Location', 'Salary', 'Summary', 'Date'])

#     for page in pages:
#         #the below source is the same as above, except it has the page reference at the end
#         #I've added {} and .format(page) so that it can reference the pages list and eventually append and loop 
#         source = requests.get('https://au.indeed.com/jobs?q=data+scientist&l=Australia&start={}'.format(page)).text

#         soup = BeautifulSoup(source, 'lxml')

#         for jobs in soup.find_all(class_='result'):
#         #first, try to return the value if present and print it out; if not, return nothing
#             try:
#                 job_title = jobs.a.text.strip()
#             except Exception as e:
#                 job_title = None
#             print('Job Title: ', job_title)

#             try:
#                 employer = jobs.span.text.strip()
#             except Exception as e:
#                 employer = None
#             print('Employer: ', employer)

#             try:
#                 location = jobs.find('span', class_='location').text.strip()
#             except Exception as e:
#                 location = None
#             print('Location: ', location)

#             try:
#                 salary = jobs.find('span', class_='salary no-wrap').text.strip()
#             except Exception as e:
#                 salary = None
#             print('Salary: ', salary)

#             try:
#                 summary = jobs.find('div', class_='summary').text.strip()
#             except Exception as e:
#                 summary = None
#             print('Summary: ', summary)

#             try:
#                 date = jobs.find('span', class_='date').text.strip()
#             except Exception as e:
#                 date = None
#             print('Date: ', date)

#             csv_print.writerow([job_title, employer, location, salary, summary, date])
            
#             print('-------------')

#             #will delay requests for X seconds per page
#             time.sleep(0.5)